탐색적 자료 분석에서 벗어나 웹 서버와 웹 서비스를 자세히 살펴본다. 이들은 어느 정도 함수의 연쇄라 할 수 있다. 웹 콘텐츠를 제공하는 문제에 대해 몇 가지 함수형 설계를 적용할 수 있다. 표현적 상태 전송(Representational State Transful, REST)에 접근하는 방법을 샆벼노는 것이다. 함수형 설계 패턴을 사용해 REST적인 웹 서비스를 구축하고 싶다.

대부분의 프레임워크에 적용할 수 있는 몇 가지 원칙을 보여줄 것이다. 함수형 디자인 패턴을 사용해 웹 콘텐츠를 제공할 수 있어야 한다. 이를 통해 함수형 설계의 장점을 살리는 웹 기반 애플리케이션을 만들 수 있다. 

예를 들어, 극단적으로 큰 데이터 집합을 살펴보거나 극단적으로 복잡한 데이터 집합을 보는 경우, 부분 집합 생성이나 검색을 지원하는 웹 서비스가 있으면 좋을 것이다. 데이터의 부분 집합을 여러 가지 형식으로 다운로드할 수 있는 웹 사이트가 필요할 수도 있다. 이렇게 더 복잡한 요구사항을 지원하는 REST적인 웹 서비스를 만들기 위해 함수형 설계가 필요할 수도 있다. 

대부분의 복잡한 웹 애플리케이션에는 사이트 사용을 쉽게 하기 위해 상태가 있는 세션이 존재한다. 세션 정보는 HTML 양식을 통해 제공한 데이터를 통해 갱신되건, 데이터베이스에서 불러오거나, 클라이언트와 서버의 이전 상호작용에 대한 캐시에서 되살릴 수 있다. 전체 상호작용에는 상태 변경이 포함되지만, 애플리케이션 프로그래밍은 상당 부분이 함수적이다. 애플리케이션의 기능 중 상당 부분은 요청 데이터나 캐시 데이터, 또는 데이터베이스 객체의 사용 여부에 대해 그리 엄격하지 않을 수 있다.

구체적인 웹 프레임워크를 자세히 알아야 할 필요를 피하기 위해 웹 서버 게이트웨이 인터페이스 디자인 패턴에 집중할 것이다. WSGI를 사용하면 간단한 웹 서버를 구현할 수 있다. 

HTTP 프로토콜을 살펴보는 것부터 시작할 것이다. 그 후 아파치 httpd 등을 사용해 프로토콜을 구현하는 방법과 mod_wsgi가 기본 서버를 확장하는 합리적인 방법인 이유를 살펴본다. 이러한 배경을 바탕으로 WSGI의 함수적인 특성을 살펴보고, 함수형 설계를 활용해 복잡한 웹 검색과 데이터 입수 도구를 구현할 수 있는지 살펴본다.

### HTTP 요청 - 응답 모델

HTTP 프로코톨의 핵심은 이상적인 경우, 상태가 없다. 사용자 에이전트 또는 클라이언트는 프로토콜에 대해 함수적인 관점을 가질 수 있따. http.client나 urllib 라이브러리를 사용하면 클라이언트를 만들 수 있다. HTTP 사용자 에이전트는 기본적으로 다음과 비슷한 기능을 실행한다.

In [ ]:
import urllib.request
with urllib.request.urlopen("http://slott-softwarearchitect.blogspot.com") as response:
    print(response.read())

wgert이나 curl과 같은 프로그램은 이러한 동작을 명령 행에서 수행한다. 이떄 인자에서 접속할 URL을 가져온다. 브라우저는 사용자의 마우스 이동과 클릭에 대한 응답으로 이러한 동작을 수행한다. 이떄 사용자의 행동, 구체적으로 말하자면 링크가 걸린 텍스트나 이미지를 클릭하는 행동으로부터 URL을 가져온다. 

하지만 서로 연결된 서버와 클라이언트 간의 상호 작용을 실용적으로 만들기 위해 상태가 있는 몇 가지 세부 구현이 생겼다. 일부 HTTP 상태 코드는 사용자 에이전트 쪽에서 추가 동작을 실행해야 함을 표현한다.

300번대 상태 코드는 요청 받은 자원이 다른 데로 이동했다는 것을 나타낸다. 그러한 응답을 받으면 사용자 에이전트는 Location 헤더의 정보에 따라 새로운 위치로 요청을 보내야 한다. 401 상태 코드는 인증이 필요하다는 뜻이다. 그러한 경우, 사용자 에이전트는 서버 접근을 위해 필요한 인증 정보가 들어간 헤더를 포함시켜 다시 요청을 보내야 한다. urllib 라이브러리에는 이러한 식의 상태가 존재하는 추가적인 부분의 구현도 들어 있다. http.client는 자동으로 300번대 위치 재지정 코드를 처리하지 않는다.

사용자 에이전트가 300번대나 401 코드를 처리하는 기법은 그렇게 샅애와 깊은 연관이 있지는 않으므로 간단한 재귀를 사용할 수 있다. 상태 코드와 위치 재지정이 아니라면, 그 경우가 기본적인 경우이며, 처리 함수는 최종 결과를 얻은 것이다. 위치 재지정이 필요하면 대상 주소를 가지고 처리 함수를 재귀호출할 수 있다. 

프로토콜의 반대쪽 끝을 보자. 정적인 콘텐츠를 제공하는 서버는 상태가 없어도 된다. HTTP 프로토콜에는 TCP/IP 소켓과 그 소켓을 활용하는 더 고수준의 HTTP구조, 이렇게 두 가지 계층이 존재한다. 저수준 처리는 socketserver 라이브러리가 담당하며, 파이썬의 http.server 라이브러리를 고수준 구현을 제공하는 여러 라이브러리 중 하나다.

http.server 라이브러리를 다음과 같이 사용할 수 있다.

In [1]:
from http.server import HTTPServer, SimpleHTTPRequestHandler
running = True

server_address = ('', 8000)

httpd = HTTPServer(server_address, SimpleHTTPRequestHandler)
while running:
    httpd.handle_request()
httpd.shutdown()

서버 객체를 만들고 httpd 변수에 그 객체를 대입했다. 외부에서 들어오는 요청을 기다릴 주소와 포트 번호를 제공했다. 해당 포트로 연결 요청이 들어오면 TCP/IP 프로토콜은 별도의 포트를 하나 할탕해 연결을 맺는다. HTTP 프로토콜은 새로 할당된 포트로부터 요청 데이터를 읽고 처리할 핸들러 handler의 인스턴스를 생성한다.

이 예제에서는 SimpleHTTPRequestHandler를 각 요청을 처리할 핸들러를 인스턴스화할 클래스로 지정했다. 이 클래스는 클라이언트에게 응답 헤더를 보낸 후에 다시 응답 본문을 보내는 최소한의 인터페이스를 구현해야 한다. 이 클래스는 로컬 디렉터리에 있는 파일을 서비스한다. 이 클래스를 변형하고 싶다면 하위 클래스를 만들고, do_GET(), do_POST() 등의 메서드를 구현하여 동작을 변경해야 한다.

종종 직접 루프를 만드는 대신 server_forever() 메서드를 사용하기도 한다. 여기서는 일반적인 경우 서버가 비정상 종료된다는 사실을 보여주기 위해 직접 루프를 만들었다. 서버를 더 부드럽게 정지시키고 싶다면, shutdown 변수의 값을 바꿀수 있는 수단이 필요하다. 예를 들어 citl+C 시그널 등을 이러한 목적으로 잘 사용한다.

### 쿠키로 상태 주입하기

쿠키가 추가되면서 클라이언트와 상태 사이의 모든 관계가 상태가 있는 것으로 바뀌었다. 흥미롭게도, HTTP 프로토콜 자체에는 변화가 없다. 상태 정보에 대한 통신은 요청과 응답 헤더를 통해 이뤄진다. 상요자 에이전트는 요청 헤더에 요청 대상 호스트와 경로에 따른 쿠키를 넣어 보낸다. 서버는 응답 헤더에 쿠키를 넣어 사용자 에이전트에게 보낸다. 

따라서 사용자 에이전트나 브라우저가 반드시 쿠기 값을 캐시해두고 요청 시 적절한 쿠키를 포함시켜야 한다. 웹 서버는 요청 헤더에서 쿠키를 받아들여야 하고, 응답 헤더에 쿠키를 넣어 전달해야 한다. 웹 서버는 쿠키를 캐시할 책임은 없다. 서버는 단지 요청의 인자 중 하나로 쿠키를 받고, 응답에 추가 내용을 더 넣을 뿐이다.

원리상 쿠키에는 아무 정보나 넣을 수 있지만, 단지 세션 상태 객체에 대한 식별자만을 포함시키는 방향으로 쿠키 사용법이 빠르게 변했다. 서버는 쿠키 정보를 사용해 세션에 대한 정보를 영속적인 저장소에서 찾을 수 있다. 이는 서버가 사용자 에이전트에 요청에 따라 세션의 상태를 변경할 수 있다는 의미인 한편, 오래된 세션을 서버가 제거할 수 있다는 의미이기도 하다. 

"세션"이라는 개념은 HTTP 프로토콜과는 별개다. 보통은 같은 세션 쿠키를 공유하는 일련의 요청을 세션이라고 정의한다. 최초 요성 시에는 쿠키가 없다. 따라서 새 세션이 만들어진다. 그 이후의 모든 요청에는 쿠키가 들어갓 것이다. 그 쿠키를 사용해 서버에 있는 세션 상태 객체를 식별할 수 있을 것이다. 이 객체에는 서버가 영속적인 웹 자료를 원활하게 제공하기 위한 정보가 들어 있을 것이다.

하지만 웹 서비스에 대한 REST 접근 방식은 쿠키에 의존하지 않는다. 각 REST 요청은 독립적이며, 전체 세션 프레임워크에 들어맞지 않는다. 이로 인해 REST적인 서비스는 사용자의 상호작용을 쿠키를 사용해 단순화하는 "사용자 친화적인" 대화식 사이트의 경우에 잘 들어맞지 않는다. 이는 또한 각 REST 요청을 별도로 인증해야 한다는 뜻이기도 하다. 보통은 서버에서 만든 간단한 토큰을 사용해, 클라이언트가 매 번 복잡한 인증 데이터를 보내지 않도록 한다. 이로 인새, 보안 소켓 계층 프로토콜을 사용해 보호하는 일이 많다. 그래서 http 대신 https를 사용한다. 

### 서버를 함수형으로 설계하기

HTTP의 핵심 아이디어 중 하나는 데몬의 응답이 요청에 의해 결정되는 함수라는 데 있다. 웹 서비스는 다음과 같이 요약할 수 있는 최상위 구현을 가져야 한다.

In [1]:
response = httpd(request)

하지만 이는 실용적이지 않다. HTTP 요청은 단순하고 큰 데이터 구조가 아니다. 실제로는 필수적인 부분과 선택적인 부분으로 이뤄진다. 요청에는 메서드와 경로가 들어간 헤더가 있으며, 본문에 입력 양식이나 업로드할 파일의 데이터 또는 그 두 가지 모두가 있을 수 있다.

좀 더 복잡하게도, 브라우저의 양식 데이터를 GET 요청의 경로 뒤에 질의 문자열로 전달할 수도 있다. 또는 POST 요청의 본문에 첨부해 보낼 수도 있다. 혼란의 여지가 있는 경우에는, 대부분의 웹 애플리케이션 프레임워크에서 form 태그에 method=poest 라는 문장의 추가하여 데이터를 헤더가 아닌 본문에 보내게 만들 것이다.

### 함수적인 관점에서 더 깊이 살펴보기

HTTP 응답과 요청에는 헤더와 본문이 이다. 요청에는 몇 가지 양식 데이터가 있을 수 있다. 따라서 웹 서버를 다음과 같이 생각할 수 있다.

In [2]:
header, content = httpd(headers, request, [uploads])

요청 헤더에는 쿠키 값이 들어갈 수 있으며, 이를 인자를 더 추가하는 것으로 생각할 수도 있다. 부가적으로, 웹 서버가 실행중인 운영체제 환경에 따라 결과가 달라지는 경우도 있다. 이러한 운영 체제 환경을 요청의 일부분으로 제공하는 인자처럼 생각할 수도 있다.

응답 내용은 매우 다양하긴 하지만, 다양한 분야의 내용물에 대한 수긍할 만한 정의가 존재한다. 다목적 인터넷 우편 확장(Multipurpose Internet Mail Extension)MIME 타입은 웹 서비스가 반환할 수 있는 내용물의 종류를 정의한다. 이러한 종류로는 일반 텍스트, HTML, JSON, XML 그리고 웹 사이트가 서비스할 수 있는 다양한 비-문자미디어 등이 있다.

HTTP 요청에 대한 응답을 만들기 위해 필요한 처리를 자세히 살펴보면, 재활용 가능한 몇 가지 공통적인 특성을 볼 수 있다. 재사용 가능한 요소라는 아이디어는 단순한 것부터 복잡한 것에 이르기까지 다양한 웹 서비스 프레임워크가 생겨나게 된 원동력이다. 함수형 설계에서는 함수를 재사용할 수 있기 때문에 함수형 접근 방식이 웹 서비스를 구현하는 데 있어 매우 적합하다고 생각된다.

서비스의 응답의 여러 요소의 파이프라인을 어떻게 만들 수 잇는지 살펴보면서 웹 서비스를 함수형으로 설계하는 방법을 본다. 요청을 처리하기 위한 함수를 내포시켜 내부에 있는 요소들이 외부의 요소들이 제공하는 부가적인 처리를 신경 쓰지 않아도 되도록 만들 것이다. 이렇게 하면 외부 요소들이 필터처럼 작동할 수 있다. 예를 들어, 외부 요소가 잘못된 요청을 오류 응답을 돌려 보내는 방식으로 처리한다면, 내부 요소들은 애플리케이션 처리에만 초점을 맞출 수 있다.

### 서비스 내포시키기

웹 요청 처리를 몇 가지 내포된 문맥으로 살펴볼 수 있다. 예로 바깥쪾 문맥에서 어떤 요청이 기존 세션에 속한 추가 요청인지, 새로운 세션의 요청인지를 검사하는 세션 관리를 담당할 수 있을 것이다. 내부 문맥에서는 크로스 사이트 요청 변조(Cross-Site Request Forgeries, CSRF)를 감지할 수 있는, 양식 처리에 필요한 토큰을 제공할 수 있을 것이다. 

앞에서 설명한 함수를 개념적으로 살펴보면 다음과 비슷한 형태가 될 것이다.

In [3]:
response = content(authentication(csrf(session(headers, request, [forms]))))

여기서 아이디어는 각 함수가 이전의 함수의 결과를 기초로 만들어 진다는 것이다. 각 함수는 요청을 더 풍부하게 하거나 잘못된 요청을 거부한다. 예를 들어 session 함수는 헤더를 사용해 요청이 기존 세션과 새 세션 중 어디에 속한 것인지 판단한다. csrf 함수는 양식 입력을 검사하여 제대로된 토큰이 들어 있는지 살펴본다. CSRF 처리를 위해서는 정상적인 세션이 필요하다. authentication 함수는 올바른 사용자 인증 정보가 없는 세션에 대해 오류 응답을 반환할 수 있고, 올바른 인증 정보가 들어온 경우에는 요청에 사용자에 대한 정보를 더 추가할 수 있다.

content 함수는 세션, 변조, 또는 인증 받지 못한 사용자에 대해 염려할 필요가 없다. 그 함수는 경로를 분석하여 어떤 콘텐츠를 제공해야 할지를 결정하는 것에만 초점을 맞춘다. 더 복잡한 애플리케이션에서는 경로 정보를 사용해 적절한 콘텐츠를 제공할 수 있는 함수를 결정하는 복잡한 매핑이 content 함수에 들어갈 수도 있다. 

하지만 내초된 함수라는 관점이 정확한 것은 아니다. 문제는 각각의 내포된 문맥이 요청을 변경할 뿐 아니라 응답도 변경해야 할 필요가 있다는 점에 있다. 실제로는 다음과 비슷한 어떤 것이 필요하다.

In [4]:
def session(headers, request, forms):
    pre-process: determine session
    content= csrf(headers, request, forms)
    post-process the content
    return the content

def csrf(headers, request, forms):
    pre-process: validate csrf tokens
    content = authenticate(headers, request, forms)
    post-processes the content
    return the content

이 개념은 웹 콘텐츠를 만들어 내기 위해 입력을 변경하거나, 출력을 변경하거나, 입출력 모두 변경하는 여러 함수의 컬렉션을 내포시켜 사용하는 함수형 설계를 보여준다. 조금만 더 생각해보면, 이러한 여러 함수들이 사용할 수 있는 간단한 표준 인터페이스를 정의할 수 있을 것이다. 인터페이스를 표준화하고 나면, 함수를 여러 방식으로 조합하고 특징을 추가할 수 있따. 웹 콘텐츠를 제공하는 간결하면서 이해하기 쉬운 프로그램을 작성할 수 있어야 한다는 함수형 프로그래밍의 목적을 달성할 수 있어야만 한다.

### WSGI 표준

WSGI는 웹 요청에 대한 응답을 만등어 내는 상대적으로 간단한 표준 설계 패턴을 정의한다. 파이썬의 wsgiref 패키지에는 WSGI의 참조 구현이 들어 있다. 

각각의 WSGI "앱"은 다음과 같은 인터페이스를 공유한다. 

In [5]:
def some_app(envison, start_response):
    return content

environ은 요청의 모든 인자를 한 균일한 구조에 담은 딕셔너리다. 헤더, 요청 메서드, 경로, 양식 데이터나 파일 업로드를 위한 첨부 등은 모두 이 환경 안에 들어있다. 이러한 정보와 더불어, 운영체제 수준의 맥락도 WSGI 요청 처리의 일부붐인 몇 가지 원소를 통해 제공된다.

start_response는 응답 상태와 헤더를 보내기 위해 반드시 사용해야 하는 함수다. 응답을 최종적으로 만드는 WSGI 서버의 일부분은 응답 텍스트를 만들어야 할 뿐 아니라 start_response 함수를 사용해 헤더와 상태를 보내야 한다. 일부 애플리케이션에서는 이 함수를 감싸 응답에 추가 헤더를 더 넣을 수 있게 할 수도 있다.

반환 값은 문자열의 시퀀스 또는 사용자 에이전트에게 돌려줄 문자열과 비슷한 파일 래퍼다. HTML 템플릿 도구를 사용한다면 이 시퀀스에 원소가 하나만 들어갈 수도 있다. Jinja2 템플릿과 같은 일부 템플릿은 템플릿을 채우는 작업을 사용자 에이전트에게 데이터를 전송하는 작업의 중간 중간에, 필요할 때마다 텍스트 덩어리의 세퀀스를 만들어 낼 수도 있다.

각각 내포되는 방식으로 인해 WSGI 애플리케이션을 연쇄적인 것으로 볼 수도 있다. 각 애플리케이션은 오류를 반환하거나 결과를 결정할 수 있는 다은 애플리케이션에게 요청을 전달할 수도 있다.

다음은 매우 단순한 라우팅 애플리케이션이다.

In [7]:
SCRIPT_MAP = {
    "demo": demo_app,
    "static": static_app,
    "": welcome_app,
}

def routing(environ, start_response):
    top_level = wsgiref.util.shift_path_info(environ)
    app = SCRIPT_MAP.get(top_level, SCRIPT_MAP[''])
    content = app(environ, start_response)
    return content

이 앱은 wsgiref.util.shift_path_info(environ)를 사용해 환경을 조작한다. 이 함수는 environ['PATH_INFO']에 있는 요청 경로를 "첫 부분/나머지 부분 분할"를 수행한다. 경로의 첫부분은 환경 environ의 SCRPT_NAME에 덧붙여 지고, PATH_INFO는 경로의 나머지 부분으로 변경된다. 그리고 반환 값은 경로의 첫 부분이다. 분석할 경로가 더 없다면 결과 값은 None이며, 환경의 내용은 변하지 않는다.

routing() 함수는 경로의 첫 번째 부분을 사용해 SCRIPT_MAP 딕셔너리에 있는 애플리케이션을 찾는다. 요청 경로를 찾지 못하는 경우에 대비해 SCRIPT_MAP['']을 기본 딕셔너리로 사용했다. 이렇게 하는 것이 404 에러 보다는 좀 더 낫다.

이 WSGI 앱은 여러 가지 다른 함수 중 하나를 선택하는 함수다. 데이터 구조 안에 정의된 함수를 평가하기 때문에 이 함수도 고차 함수라고 생각할 수 있다. 

정규식을 사용했더라면 프레임워크가 경로 매치 과정을 더 쉽게 일반화할 수 있었다는 것을 알 수 있다. routing() 함수가 문자열이 아니라 정규식과 WSGI 애플리케이션을 연관시켜주는 딕셔너리를 사용하고, 경로를 정규식을 사용해 매치시키게 만들 수도 있다. 이렇게 개선한 routing 함수를 사용한 애플리케이션에서는 각 정규식이 일치하는지 검사할 수 있다. 일치하는 정규식을 찾았다면, 요청이 지정한 애플리케이션을 호출하기 전에 groups() 함수를 사용해 환경 정보를 변경할 수 있을 것이다.

### WSGI 처리 도중에 예외 발생시키기

WSGI 애플리케이션의 핵심적인 특징 중 하나는 연쇄상의 각 단계에 요청을 걸러낼 책임이 있다는 것이다. 가능하면 잘못된 요청을 빨리 거부하자는 것이 그 아이디어다. 파이썬의 예외 처리를 활용하면 이러한 경우를 특히 단순하게 처리할 수 있다. 

정적인 콘텐츠를 제공하는 WSGI 애플리케이션을 다음과 같이 정의할 수 있다.

In [8]:
def static_app(environ, start_response):
    try:
        with open(CONTENT_HOME + envision["PATH_INFO"]) as static:
            content = static.read().encode('utf-8')
            headers = [ ('Content-Type', 'text/plain; charset="utf-8"'), ('Content-Length', str(len(content)))]
            start_response('200 OK', headers)
            return [content]
    except IsADirectoryError as e:
        return index_app(environ, start_response)
    except FileNotFoundError as e:
        start_response('404 NOT FOUND', [])
        return([repr(e).encode('utf-8')])

이 경우, 요청 받은 경로를 텍스트 파일로 취급해 열었다. 해당 경로의 파일을 열 수 없는 일반적인 이유는 크게 두 가지다. 그 두 경우를 모두 예외로 처리했다.

다른 모든 예외는 처리하지 않는다. 이 애플리케이션을 호출한 애플리케이션에서는 더 일반적인 오류 응답 기능이 들어있어야만 한다. 그러한 애플리케이션에서도 예외를 처리하지 않는다면, WSGI의 일반적인 실패 응답이 사용된다.

더 나아가, 콘텐츠를 바이트로 제공해야 한다. 이는 파이썬 문자열을 정상적으로 인코딩해야 하며, 인코딩 정보를 사용자 에이전트에 제대로 전달해야 한다는 뜻이다. 심지어는 오류 메세지인 repr(e) 조차도 사용자 에이전트에게 전달하기 전에 제대로 인코딩해야 한다.

### 실용적인 WSGI 애플리케이션

WSGI 표준의 의도는 완전환 웹 프레임워크를 정의하는 것이 아니다. 진정한 의도는 웹과 관련 있는 여러 처리를 유연하게 연동할 수 있는 최소한의 표준을 만드는 것이다. 프레임워크는 웹 서비스를 제공하기 위한 내부 구조와는 매우 다른 접근 방법을 선택할 수도 있다. 하지만 그 가장 바깥쪽 인터페이스가 WSGI와 같아야만 여러 가지 문맥에서 사요될 수 있다.

아파치 httpd나 엔진 X 등의 웹 서버에는 웹 서버에서 파이썬 애플리케이션으로 연결해주는 WSGI와 호환되는 인터페이스를 제공하는 어댑터가 있다. 

애플리케이션을 더 큰 서버에 포함시키면 관심사를 깔끔하게 분리할 수 있다. 아파치 httpd를 사용하여, css js 이미지 파일 드으이 완전히 정적인 콘텐츠를 서비스할 수 있다. 하지만 HTML 페이지는 아파치의 mod_wsgi 모듈을 사용해 별도의 파이썬 프로세스에게 작업을 맡길 수 있따. 그 파이썬 프로세스는 웹 콘텐츠에서 관심 있는 부분의 HTML만을 처리할 것이다.

이렇게 할 수 있다는 것은 별도의 미디어 서버를 만들거나 웹 사이트의 경로를 두가지 부분으로 나눠야 한다는 뜻이다. 두 번째 방식을 선택한다면, 일부 경로는 완전히 정적인 콘텐츠만을 제공하며, 아파치 httpd가 이를 처리할 것이다. 다른 경로에는 동적인 콘텐츠가 들어가고, 파이썬이 그 경로를 처리할 것이다. 

WSGI 기능을 사용하는 경우, 외부의 WSGI 인터페이스를 어떤 방법으로도 바꿀 수 없다는 것을 기억해야 한다. 예를 들어, 요청을 처리하는 일련의 함수의 시퀀스인 매개변수를 추가하는 것이 더 좋은 경우가 있을 것이다. 각 처리 단계에서는 그 함수의 리스트에서 첫 번째 원소를 가져와 다음 처리를 수행할 수 있을 것이다. 이와 같이 매개변수를 추가하는 일이 함수형 프로그래밍에서는 흔한 일이지만, 이렇게 인터페이스를 변경하는 것은 WSGI의 목적에 위배된다.

WSGI 정의로 인해 전역 변수나 요청 환경, 또는 캐시에서 전역 설정 객체를 가져오는 함수를 통해서만 성정이 이뤄져야 한다. 작은 예제에서는 모듈 수준의 전역 변수를 사용하는 것도 가능하다. 더 복잡한 애플리케이션에서는 아마 설정 캐시가 필요할 것이다. 또한 environ 딕셔너리만에 있는 설정 정보만을 변경하고, 다른 WSGI 애플리케이션에게 처리에 대한 제어를 위임하는 별도의 WSGI 앱을 만드는 것도 타당하다. 

### 웹 서비스를 함수로 정의하기

원본 데이터를 잘라내서 흔든 후 JSON, XML, CSV 형식으로 다운로드 할 수 있는 REST적인 웹 서비스를 살펴본다. 서비스 전체에 대한 WSGI 래퍼를 제공하지만 이 애플리케이션의 실제 작업을 수행하는 함수는 WSGI의 제약을 따르지는 않을 것이다.

네 가지 하위 컬렉션이 있는 안스콤 쿼텟을 데이터 집합으로 사용한다. 

애플리케이션은 2 계층으로 나눈다. 웹 계층은 간단한 WSGI 애플리케이션이며, 나머지 처리 부분은 좀더 전형적인 함수형 프로그래밍을 사용할 것이다. 의미가 있는 결과를 제공하기 위한 함수적 접근 방식에 초점을 맞추기 위해 웹 계층을 먼저 살펴본다. 

웹 서비스에 두 가지 정보를 제공할 필요가 있다.

* 원하는 쿼텟 - 이 부부은 잘라내서 흔들기에 해당한다. 예제는 잘라내기다.

* 원하는 출력 형식

데이터 선택에는 보통 경로를 사용한다. 핵심 아이디어는 URL이 자원을 정의하며, 그 URL을 변경할 만한 이유는 없다는 것이다. 여기서는 데이터 집합이나 날짜나 조직 내에서의 허가 여부, 또는 다른 외부 요인과는 무관하다. 따라서 URL은 시간과 관계가 없고, 절대적이다.

출력 형식은 URL을 구성하는 핵심적인 부분은 아니다. 단지 직렬화 형식일 뿐이며, 데이터 자체는 아니기 때문이다 경우에 따라 HTTP Accept 헤더를 사용해 형식을 지정하기도 한다. 그러한 방식을 브라우저에서 사요하는 것은 어렵지만, REST적인 API를 사용하는 애플리케이션에서는 사용하기 쉽다. 데이터를 브라우저를 사용해 꺼내는 경우에는 보통 질의 문자열을 사용해 출력 형식을 지정한다. form=json과 같은 문자열을 경로 뒤에 붙이는 방식을 사용하여 출력 형식을  JSON으로 지정할 수 있다. 

URL은 다음과 같을 것이다.

http://localhost:8080/anscombe/lll/?form=csv

이 URL은 3번째 데이터 집합을 CSV 형식으로 다운로드 한다.

### WSGI 애플리케이션 만들기

간단한 URL 패턴 매치 식을 사용해 이 애플리케이션이 처리할 유일한 라우팅을 정의한다. 더 복잡한 애플리케이션은 여럿 패턴이 존재할 것이다.

In [9]:
import re
path_pat = re.compile(r"^/anscombe/(?P<dataset>.*?)/?$")

이러한 선택에 이름 있는 매개변수를 사용한다. 많은 경우, REST적인 API를 다음과 비슷한 문법을 사용해 기술한다.

/anscombe/{dataset}/

이러한 이상적인 패턴을 적절한 정규식으로 바꾸고, 경로의 데이터 집합 선택자의 이름을 유지한다. 

다음은 이러한 패턴이 어떻게 작동하는지를 보여주는 단위 테스트다.

In [10]:
test_pattern = """"""
m1= path_pat.match('/anscombe/I')

In [11]:
m1.groupdict()

{'dataset': 'I'}

In [12]:
m2= path_pat.match('/anscombe/II')
m2.groupdict()

{'dataset': 'II'}

In [13]:
m3= path_pat.match('/anscombe/')
m3.groupdict()

{'dataset': ''}

방금 본 세 가지 테스트를 전체 독테스트에 포함시킬 수 있다.

In [16]:
__test__ = {
    "test_pattern":test_pattern
}

나머지 WSGI 애플리케이션과 별도로 테스트할 수 있다는 사실이 중요하다. 

다음은 전체 WSGI 애플리케이션이다.

In [17]:
import traceback
import urllib

def anscombe_app(environ, start_response):
    log = environ['wsgi.errors']
    try:
        match = path_pat.match(environ['PATH_INFO'])
        set_id = match.group('dataset').upper()
        query = urllib.parse.parse_qs(environ['QUERY_STRING'])
        print(environ['PATH_INFO'], environ['QUERY_STRING'], match.groupdict(),file=log)
        log.flush()
        dataset = anscombe_filter(set_id, raw_data())
        content, mime = serialize(query['form'][0], set_id, dataset)
        headers = [
            ('Content-Type', mime),
            ('Content-Length', str(len(content))),]
        start_response("200 OK", headers)
        return [content]
    except Exception as e:
        traceback.print_exc(file=log)
        tb = traceback.format_exc()
        page = error_page.substitute(title="ERROR", message=repr(e), traceback=tb)
        content = page.encode('utf-8')
        headers = [
            ('Content-Type', "text/html"),
            ('Content-Length', str(len(content))),
        ]
        start_response("404 NOT FOUND", headers)
        return [content]

PATH_INFO: 뽑아낼 데이터
QUERY_STRING: 출력 형식

raw_data(): 파일에서 원 데이터를 읽음. 결과는 Pair 객체의 리스트가 있는 딕셔너리다.
anscombe_filter(): 선택 기준 문자열과 원데이터 딕셔너리를받아 Pair 객체의 리스트를 반환.
serialize(): 반환된 리스트를 바이트로 변환 

Content-Length 헤더: 각 직렬화기를 바이트를 내놓는 제너레이터 함수로 바꿀 수 있음 데이터의 집합이 큰 경우 이렇게 하는 것이 최적화에 도움이 된다. 하지만 다운로드를 진행하는 사용자의 입장에서는 브라우저가 다운로드가 얼마나 완료됐는지 표시해주지 않기 때문에 기분지 좋지는 않을 것이다. 

디버깅을 위해 결과 웹 페이즈에 파이선 스택 트레이스를 표시. 그러나 API가 제공하는 피트백은 요청을 수정하기에만 충분하면 되며 그 이상을 들어가서는 안된다. 너무 많은 정보를 사용자에게 제공함.

### 원데이터 얻기

In [21]:
from collections import namedtuple

Pair = namedtuple("Pair", ("x", "y"))
def series(n, row_iter):
    for row in row_iter:
        yield Pair(*row[n*2:n*2+2])

def row_iter(source):
    return csv.reader(source, delimiter="\t")

def head_split_fixed(row_iter):
    title = next(row_iter)
    assert len(title) == 1 and title[0] == "Anscombe's quartet"
    heading = next(row_iter)
    assert len(heading) == 4 and heading == ["I", "II", "III", "IV"]
    columns = next(row_iter)
    assert len(columns) == 8 and columns == ['x', 'y', 'x', 'y', 'x', 'y', 'x', 'y']
    return row_iter

def raw_data():
    with open("Anscombe.txt") as source:
        data = tuple(head_split_fixed(row_iter(source)))
        mapping = dict((id_str, tuple(series(id_num, data))) for id_num, id_str in enumerate(['I', 'II', 'III', 'IV']))
    return mapping

row_iter: 파일을 구문 분석하여 열이 구분된 줄을 반환
head_split_fixed: 파일에서 표의 머릿줄 부분 삭제
series: 데이터 집합의 x, y 쌍으로부터 Pair 객체 생성

여기서 바꾼 중요한 것은, 독 테스트에 테스트 케이스를 포함시켰다는 것이다. 앞에서 설명했지만, 웹 애플리케이션을 전체적으로 테스트하기는 어렵다. 웹 서버를 시작하고, 웹 클라이언트를 사용해 테스트 케이스를 실행해야만 한다. 그 후 웹 로그를 확인해 문제를 해결해야 하는데, 완전한 스택 트레이스가 없다면 그 또한 어려울 수 있다. 따라서 웹 애플리케이션에서 가능한 많은 부분을 일반적인 독테스트나 단위테스트기법을 활용해 테스트하는 것이 좋다. 

### 필터 적용하기

이 앱은 매수 단순한 필터를 걸러낸다. 전체 과정은 다음 함수에 들어 있다.

In [23]:
def anscombe_filter(set_id, raw_data):
    """
    >>> anscombe_filter("II", raw_data()) $ doctest:
    (Pair(x=10.0, y=9.14), Pair(x=8.0, y=8.14), Pair(x=13.0, y=8.74), ...)
    """
    return raw_data[set_id]

단순한 식을 만든 이유

* 함수를 사용하는 편이 더 일관성이 있고, 첨자 식을 사용하는 것보다 좀 더 유연하다.

* 함수로 만들면 필터를 쉽게 확장할 수 있다.

* 이 함수에 대한 독스트링에 별도의 단위 테스트를 포함시킬 수 있다.

단순한 람다도 잘 작동하겠지만, 테스트하기에는 그리 편리하지 않다. 예외는 처리하지 않았다. 함수에 문제가 발생하면 예외가 던져질 것이다. WSGI 래퍼 함수는 모든 예외를 잡아 적절한 상태 정보와 오류 응답을 클라이언트에게 보내야 한다.

set_id가 잘못된 경우, 그것에 집착하는 대신 그냥 파이썬의 예외를 던지게 내버려뒀다. 실제로 이 함수는 허가를 구하는 것보다는 요서를 바라는 편이 더 낫다라는 파이썬의 충고를 따른다. 이 함수 코드에서는 허가를 구하는 것을 회피해야 한다는 충고를 인자가 정상인지 검사하는 if문을 없앰으로써 실제로 구현했다. 이 코드에는 오직 용서를 처리하는 부분만 존재한다. 예외를 던져 WSGI 래퍼에게 처리하는 부분이 바로 그것이다. 이 중요한 충고를 앞에서 본 원데이터 처리와 이제 보게 될 직렬화에서도 적용했다.

### 결과 직렬화하기 

직렬화란, 파이썬 데이터를 전송에 적당한 바이트의 스트림으로 바꾸는 것이다. 각각의 형식은 한 가지 형식만을 직렬화할 수 있는 간단한 함수로 표현할 수 있다. 최상위의 일반적인 직렬화기는 여러 가지 구체적인 직렬화기 중에서 원하는 것으 선택한다. 직렬화를 선택하는 것은 다음과 같은 함수로 정리할 수 있다.

In [25]:
serializers = {
    'xml': ('application/xml', serialize_xml),
    'html': ('text/html', serialize_html),
    'json': ('application/json', serialize_json),
    'csv': ('text/csv', serialize_csv),
}

def serialize(format, title, data):
    mime, function = serializers.get(format.lower(), ('text/html', serialize_html))
    return function(title, data), mime

직렬화기를 문자열을 만들어 내는 것과 바이트를 만들어 내는 두 그룹으로 나눌 수 있다. 문자열을 만들어 내는 직렬화기는 문자열을 바이트로 인코딩해야 한다. 바이트를 만들어 내는 직렬화기는 더 이상의 잡업이 필요없다.

문자열을 만드는 직렬화기의 경우, 직렬화기를 바이트로 변환하는 표준 변환가 함수 합성해야 한다. 데코레이터를 사용해 함수적 합성이 가능하다.

In [29]:
from functools import wraps

def to_byte(function):
    @wraps(function)
    def decorated(*args, **kw):
        text = function(*args, **kw)
        return text.encode('utf-8')
    return decorated



to_byte는 주어진 함수를 평가한 결과를 UTF-8 인코딩을 사용한 바이트로 바꾼다. 이 데코레이터를 JSON, CSV, HTML 직렬화기와 사용하는 방법을 보여줄 것이다. 

In [31]:
serializers = {
    'xml': ('application/xml', serialize_xml),
    'html': ('text/html', to_byte(serialize_html)),
    'json': ('application/json', to_byte(serialize_json)),
    'csv': ('text/csv', to_byte(serialize_csv)),
}

### 데이터를 JSON이나 CSV로 직렬화하기

다음은 JSON 직렬화기다.

In [32]:
import json
@to_byte
def serialize_json(series, data):
    obj = [dict(x=r.x, y=r.y) for r in data]
    return json.dumps(obj, sort_keys=True)

다음은 CSV 직렬화기다.

In [33]:
import csv, io
@to_byte
def serialize_csv(series, data):
    buffer = io.StringIO()
    wtr = csv.DictWriter(buffer, Pair._fields)
    wtr.writeheader()
    wtr.writerow(r._asdict() for r in data)
    return buffer.getvalue()

CSV 모듈의 리더와 라이터는 명령형과 함수형 요소를 함께 사용한다. 

### 데이터를 XML로 직렬화하기

다음은 XML 직렬화기다.

In [34]:
import xml.etree.ElementTree as XML
def serialize_xml(series, data):
    doc = XML.Element("series", name=series)
    for row in data:
        row_xml = XML.SubElement(doc, "row")
        x = XML.SubElement(row_xml, "x")
        x.text = str(row.x)
        y = XML.SubElement(row_xml, "y")
        y.text= str(row.y)
    return XML.tostring(doc, encoding='utf-8')

ElementTree는 명령형인 경향이 있다. 이로 인해 다른 함수형 설계와 잘 들어맞지 않는다. 명령형 스타일 외에도, DTD나 XSD를 만들지 않았다는 사실을 기억해야 한다. 태그에 제대로 이름 공간을 할당하지 않았따. 또한 일반적으로 XML 문서의 첫 번째 부분에 들어가는 처리 명령도 추가하지 않았다. 더 복잡한 직렬화 라이브러리가 도움이 될 것이다. wiki.python.org/moin/PythonXML에서 대안을 찾아보자.

### HTML 직렬화하기

마지막으로 HTML 문서를 만드는 것이 얼마나 복잡한지 보여줄 것이다. 이러한 복잡도는 HTML에서는 전체 웹 페이지를 일정한 문맥 정보 안에 위치시켜야 한다는 사실에서 비롯된 것이다.

In [36]:
import string
data_page = string.Template("""<html>
<head><title>Series ${title}</title></head>
<body><h1>Serise ${title}</h1>
<table><thead><tr><td>x</td><td>t</td></tr></thead>
<tbody>
${rows}
</tbody></table></body></html>""")

@to_byte
def seriailize_html(series, data):
    return data_page.substitute(title=series, rows="\n".join("<tr><td>{0.x}</td><td>{0.y}</td></tr>".format(row) for row in data))

복잡한 결과 집합을 사용해야 하는 경우 이어란 방식은 적합하지 않다. HTML 페이지를 만들 수 있는 복잡한 템플릿 도구가 많이 있다. 이 중 상당수는 직렬화를 시작하는 함수와 별도로 템플릿 안에 루프를 내포시킬 수 있는 기능이 있다. wiki.python.org/moin/Templating 에서 찾아보자. 

### 사용량 추적하기

API키는 접근을 위해 사용된다. 인증이나 허가를 위해 사용한다. 사용량 추적에도 ㅅ하용된다. 너무 많이 사용하면 거부한다. 
API키를 만드느 한 가지 방법은 안전한 난수를 사용해 예측하기 어려운 키 문자열을 만드는 것이다. 다음과 같이 작은 함수만으로도 충분할 수 있다.

In [38]:
import random
rng = random.SystemRandom()
import base64
def make_key(rng=rng, size=1):
    key_bytes = bytes(rng.randrange(0, 256) for i in range(18*size))
    key_string = base64.urlsafe_b64encode(key_bytes)
    return key_string

어떤 난수 바이트에 대해 베이스 64 인코딩을 사용해 문자의 시퀀스를 만들었다. 난수 바이트의 바이트 개수가 3의 배수가 되도록 만들면, 베이스 64 인코딩의 뒤에 '='가 오는 것을 피할 수 있다. URL에서 안전하게 사용할 수 있는 베이스 64 인코딩을 사용했기 때문에 문자열 안에는 URL이나 질의 문자열에 들어갈 수 없는 '/'나 '+'가 들어 있지 않다. 

난수 키의 개수는 다음과 같다.

In [39]:
2**(18*8)

22300745198530623141535718272648361505980416

다른 방법은 uuid.uuid4()를 사용해 임의의 고유 식별자 UUID(Universally Unique Identifier)를 만드는 것이다. 32개의 16진수와 4개의 '-'가 들어간 36 문자로된 문자열이다. UUID는 변조하기 어렵다. 사용자 이름이나 호스트 IP 주소 등을 포함하는 UUID를 사용하는 것은, 인코딩한 정보를 포함하고 있기 좋은 생각이 아니다. 그 정보를 디코딩하여 위조를 위한 키로 사용할 수도 있다. 

REST적인 웹 서버에서는 정당한 키와 클라이언트 연락처 정보가 들어간 작은 데이터베이스가 필요할 것이다. 어떤 API 요청이 그 데이터베이스에 있는 키를 포함한다면, 데이터베이스에서 찾은 사용자가 해당 요청을 보낸 것이다. API 요청에 알려진 키가 들어있지 않으면, 요청을 거부하고 401 응답을 보낼 수 있다. 24문자로된 문자열이기 때문에 상당히 작고 캐시할 수 있다. 일반적인 로그 분석으로도 어떤 키의 사용량을 충분히 알 수 있을 것이다. 더 복잡한 애플리케이션이라면 API 요청을 별도의 로그파일이나 데이터베이스에 기록하여 분석을 쉽게 할 수 있다.